In [1]:
import kagglehub
import numpy as np
import pandas as pd
from collections import Counter
from sklearn.metrics import accuracy_score
# Download latest version
path = kagglehub.dataset_download("wenruliu/adult-income-dataset")


D:\Programowanie\deep_learning\.venv1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = (pd.read_csv(path+"\\adult.csv",na_values="?",skipinitialspace=True)
        .dropna()
        )

for col in df.select_dtypes(include='object'):
    df[col] = df[col].str.strip()


In [3]:
df_train = df.iloc[:30162]
df_test = df.iloc[30162:]
x_train = df_train
y_train = df_train["income"]
x_test = df_test.drop(columns="income")
y_test = df_test["income"]

In [6]:
probability = dict(y_train.value_counts(normalize=True))
features = [col for col in x_train.columns if col not in ["income"]]


In [7]:
all_counts = {}
print()
for feature in features:
    counts = x_train.groupby('income')[feature].value_counts().unstack(fill_value=0)
    smoothed = counts+1
    smoothed_probs = smoothed.div(smoothed.sum(axis=1)+smoothed.shape[1],axis=0)
    all_counts[feature] = smoothed_probs
print(all_counts)


{'age': age           17        18        19        20        21        22        23  \
income                                                                         
<=50K   0.013858  0.019891  0.024525  0.027322  0.026448  0.030557  0.035541   
>50K    0.000000  0.000000  0.000264  0.000132  0.000792  0.001187  0.001715   

age           24        25        26  ...        80        81        82  \
income                                ...                                 
<=50K   0.032262  0.030383  0.028634  ...  0.000743  0.000743  0.000262   
>50K    0.004486  0.006597  0.005806  ...  0.000396  0.000528  0.000000   

age           83        84        85        87        88        89        90  
income                                                                        
<=50K   0.000262  0.000262  0.000087  0.000044  0.000087  0.000044  0.000962  
>50K    0.000000  0.000132  0.000132  0.000000  0.000132  0.000000  0.001451  

[2 rows x 73 columns], 'workclass': workclass  Feder

In [12]:
predictions = []
for i, row in x_test.iterrows():
    scores = {}
    for c in ['<=50K', '>50K']:
        log_prob = np.log(probability[c])
        for feature in features:
            value = row[feature]
            prob = all_counts[feature].loc[c, value]
            log_prob += np.log(prob)
        scores[c] = log_prob
    predicted_class = max(scores, key=scores.get)
    predictions.append(predicted_class)

KeyError: 236497